# COS60008 Introduction to Data Science
## Assignment 3
## Student Details:
* Name: Kan Asvasena
* Student ID:
* Email:
* Submission Date: 23 June 2023
* TuteLab Class:

## Task 3 – Test & Train Data Preparation (5%)

### Import Dependencies, Settings, and Load Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_colwidth', None)
%config InlineBackend.figure_format='retina'

data = pd.read_csv('../data/processed/clean_data.csv')

In [2]:
data.head()

,Year,Location,Avg Min Temp *C,Yr Rain mm,Heat Wave,Dry Spell,Cold Wave,Wet Spell,Irrigation,Crop Type,Crop Damage,Yield,Observer
0,1980,Emilyland,19.6,707.94118,No,No,Yes,No,Yes,Wheat,Yes,1287.55,Ryan Stewart
1,1980,East Xavierport,23.0,615.49026,No,No,Yes,No,Yes,Wheat,Yes,1426.41,Tracie Bell
2,1980,North Daniel,21.0,1230.83066,No,No,No,No,No,Maize,No,796.92,Thomas Miller
3,1981,Emilyland,19.9,852.34018,No,No,Yes,No,Yes,Wheat,Yes,1305.04,Sara Nguyen
4,1981,East Xavierport,23.2,591.44154,No,No,Yes,No,Yes,Maize,Yes,1031.42,Tracie Bell


### 3.1 Dropping Non-relevant Feature
Drop the 'Observer' column as it is not relevent to the prediction.

In [3]:
# drop Observer column
data.drop('Observer', axis=1, inplace=True)
data.head(10)

,Year,Location,Avg Min Temp *C,Yr Rain mm,Heat Wave,Dry Spell,Cold Wave,Wet Spell,Irrigation,Crop Type,Crop Damage,Yield
0,1980,Emilyland,19.6,707.94118,No,No,Yes,No,Yes,Wheat,Yes,1287.55
1,1980,East Xavierport,23.0,615.49026,No,No,Yes,No,Yes,Wheat,Yes,1426.41
2,1980,North Daniel,21.0,1230.83066,No,No,No,No,No,Maize,No,796.92
3,1981,Emilyland,19.9,852.34018,No,No,Yes,No,Yes,Wheat,Yes,1305.04
4,1981,East Xavierport,23.2,591.44154,No,No,Yes,No,Yes,Maize,Yes,1031.42
5,1981,North Daniel,21.0,1337.11950,No,No,No,No,No,Soy,No,1497.04
6,1982,Emilyland,19.9,762.67056,No,No,Yes,No,Yes,Soy,Yes,1601.38
7,1982,East Xavierport,23.1,678.40098,No,No,Yes,No,Yes,Soy,Yes,1727.60
8,1982,North Daniel,21.1,1378.86186,No,No,No,No,No,Soy,No,1496.62
9,1983,Emilyland,20.3,810.13046,No,No,Yes,No,Yes,Maize,Yes,923.89


### 3.2 Conversion from Categorical to Numerical

In [4]:
le = LabelEncoder()
mapping = {}

# For binary categories, we can use Label Encoding
binary_categories = ['Heat Wave', 'Dry Spell', 'Cold Wave', 'Wet Spell', 'Irrigation', 'Crop Damage']

for column in binary_categories:
	data[column] = le.fit_transform(data[column])
	mapping[column] = dict(zip(le.classes_, le.transform(le.classes_)))

# For non-binary categorical features, we use One-Hot Encoding
data = pd.get_dummies(data, columns=['Location', 'Crop Type'])

mapping_df = pd.DataFrame(mapping)
mapping_df.fillna('-', inplace=True)
print('Mapping of categorical features:')
display(mapping_df)
display(data.head(10))

Mapping of categorical features:


,Heat Wave,Dry Spell,Cold Wave,Wet Spell,Irrigation,Crop Damage
No,0,0,0,0,0,0
Yes,1,1,1,1,1,1


,Year,Avg Min Temp *C,Yr Rain mm,Heat Wave,Dry Spell,Cold Wave,Wet Spell,Irrigation,Crop Damage,Yield,Location_East Xavierport,Location_Emilyland,Location_North Daniel,Crop Type_Maize,Crop Type_Soy,Crop Type_Wheat
0,1980,19.6,707.94118,0,0,1,0,1,1,1287.55,0,1,0,0,0,1
1,1980,23.0,615.49026,0,0,1,0,1,1,1426.41,1,0,0,0,0,1
2,1980,21.0,1230.83066,0,0,0,0,0,0,796.92,0,0,1,1,0,0
3,1981,19.9,852.34018,0,0,1,0,1,1,1305.04,0,1,0,0,0,1
4,1981,23.2,591.44154,0,0,1,0,1,1,1031.42,1,0,0,1,0,0
5,1981,21.0,1337.11950,0,0,0,0,0,0,1497.04,0,0,1,0,1,0
6,1982,19.9,762.67056,0,0,1,0,1,1,1601.38,0,1,0,0,1,0
7,1982,23.1,678.40098,0,0,1,0,1,1,1727.60,1,0,0,0,1,0
8,1982,21.1,1378.86186,0,0,0,0,0,0,1496.62,0,0,1,0,1,0
9,1983,20.3,810.13046,0,0,1,0,1,1,923.89,0,1,0,1,0,0


### 3.3 Normalisation

In [5]:
# Define the columns you want to scale
columns_to_scale = ['Year', 'Avg Min Temp *C', 'Yr Rain mm', 'Heat Wave', 'Dry Spell', 'Cold Wave', 
                    'Wet Spell', 'Irrigation', 'Crop Damage', 
                    'Location_East Xavierport', 'Location_Emilyland', 'Location_North Daniel',
                    'Crop Type_Maize', 'Crop Type_Soy', 'Crop Type_Wheat']

# Scale these columns using MinMaxScaler
scaler = MinMaxScaler()
scaled_columns = pd.DataFrame(scaler.fit_transform(data[columns_to_scale]), columns=columns_to_scale, index=data.index)

# Drop original columns
data.drop(columns=columns_to_scale, inplace=True)

# Merge the scaled columns with the original data
data = data.merge(scaled_columns, left_index=True, right_index=True)

data.head(10)


,Yield,Year,Avg Min Temp *C,Yr Rain mm,Heat Wave,Dry Spell,Cold Wave,Wet Spell,Irrigation,Crop Damage,Location_East Xavierport,Location_Emilyland,Location_North Daniel,Crop Type_Maize,Crop Type_Soy,Crop Type_Wheat
0,1287.55,0.000000,0.000000,0.077331,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1426.41,0.000000,0.596491,0.015963,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,796.92,0.000000,0.245614,0.424418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1305.04,0.025641,0.052632,0.173181,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1031.42,0.025641,0.631579,0.000000,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
5,1497.04,0.025641,0.245614,0.494971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
6,1601.38,0.051282,0.052632,0.113660,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
7,1727.60,0.051282,0.614035,0.057723,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
8,1496.62,0.051282,0.263158,0.522679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9,923.89,0.076923,0.122807,0.145163,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


### 3.4 Splitting

In [6]:
a_X = data.drop('Yield', axis=1) # Features
a_y = data['Yield'] # Target variable

b_X = data.drop('Crop Damage', axis=1) # Features
b_y = data['Crop Damage'] # Target variable

In [7]:
test_size = 0.2
random_state = 42


a_X_train, a_X_test, a_y_train, a_y_test = train_test_split(a_X, a_y, test_size=test_size, random_state=random_state)
b_X_train, b_X_test, b_y_train, b_y_test = train_test_split(b_X, b_y, test_size=test_size, random_state=random_state)

### 3.5 Save

In [8]:
path = '../data/processed/'

a_X_train.to_csv(path + 'a_X_train.csv', index=False)
a_X_test.to_csv(path + 'a_X_test.csv', index=False)
a_y_train.to_csv(path + 'a_y_train.csv', index=False)
a_y_test.to_csv(path + 'a_y_test.csv', index=False)

b_X_train.to_csv(path + 'b_X_train.csv', index=False)
b_X_test.to_csv(path + 'b_X_test.csv', index=False)
b_y_train.to_csv(path + 'b_y_train.csv', index=False)
b_y_test.to_csv(path + 'b_y_test.csv', index=False)
